In [32]:
import json
from nltk.tokenize import sent_tokenize
import re

In [12]:
path = '../classification-scripts/same-noun-modifications/SAME-NOUN-MODIFICATIONS-TEST-5-NEW.json'
with open(path, 'r') as json_in: 
     data = json.load(json_in)

In [30]:
def sentence_splitter(document):
    """
      Sentence splitter to deal with bullet items in texts.
    """
    # Tokenize per 'sub sentence list' instead of joining (to keep markdown headers separated)
    unflattened_sentences = (sent_tokenize(sent_item)
                             for sent_item in document)
    # Flatten sentences: (https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists)
    sentences = (
        sentence for sub_sentences in unflattened_sentences for sentence in sub_sentences)
    pattern = re.compile(r"^[0-9]+\.$")
    merged_item_sents = []

    sentence = next(sentences)
    while sentence:
        if re.match(pattern, sentence):
            next_sentence = next(sentences)
            merged = f"{sentence} {next_sentence}"
            merged_item_sents.append(merged)
        else:
            merged_item_sents.append(sentence)

        try:
            sentence = next(sentences)
        except StopIteration:
            sentence = False
    return merged_item_sents

In [13]:
data[0].keys()

dict_keys(['Filename', 'All_Versions', 'Revision_Length', 'Base_Sentence', 'Revisions', 'Key', 'Source_Tokenized', 'Target_Tokenized', 'Correction', 'Source_Line_Tagged', 'Target_Line_Tagged', 'Differences', 'Entailment_Rel', 'PPDB_Matches', 'Loc_in_splits', 'Target_Line_Nr', 'Source_Line_Nr', 'Source_Context', 'Target_Context', 'Source_Context_5', 'Target_Context_5', 'Source_Context_5_Processed', 'Target_Context_5_Processed'])

In [27]:
context_example = data[10]['Source_Context_5_Processed']

In [16]:
context_example

'To find an appraiser who might be able to train you, run a search online for “real estate appraisers” or some similar formulation. Alternately, use your phone book to locate real estate appraisers. * In some cases, your local licensing authority will connect you with an appraiser. * You can often begin as an apprentice before beginning your educational requirements, but you won’t be able to complete your apprenticeship and become a licensed real estate appraiser without gaining the requisite education. * In the U.S., you could use the Appraisal Subcommittee’s “Find an appraiser” search function at https://www.asc.gov/Pages/FindAnAppraiser.aspx. 3. Complete the real estate appraisal application. The application varies from country to country, and regional variations often exist even within a single nation. Generally, however, you’ll need to include (or agree to) a criminal background check, provide copies of your academic transcripts, and provide basic contact information (including yo

In [8]:
from nltk.tokenize import word_tokenize

In [17]:
word_tokenize(context_example)

['To',
 'find',
 'an',
 'appraiser',
 'who',
 'might',
 'be',
 'able',
 'to',
 'train',
 'you',
 ',',
 'run',
 'a',
 'search',
 'online',
 'for',
 '“',
 'real',
 'estate',
 'appraisers',
 '”',
 'or',
 'some',
 'similar',
 'formulation',
 '.',
 'Alternately',
 ',',
 'use',
 'your',
 'phone',
 'book',
 'to',
 'locate',
 'real',
 'estate',
 'appraisers',
 '.',
 '*',
 'In',
 'some',
 'cases',
 ',',
 'your',
 'local',
 'licensing',
 'authority',
 'will',
 'connect',
 'you',
 'with',
 'an',
 'appraiser',
 '.',
 '*',
 'You',
 'can',
 'often',
 'begin',
 'as',
 'an',
 'apprentice',
 'before',
 'beginning',
 'your',
 'educational',
 'requirements',
 ',',
 'but',
 'you',
 'won',
 '’',
 't',
 'be',
 'able',
 'to',
 'complete',
 'your',
 'apprenticeship',
 'and',
 'become',
 'a',
 'licensed',
 'real',
 'estate',
 'appraiser',
 'without',
 'gaining',
 'the',
 'requisite',
 'education',
 '.',
 '*',
 'In',
 'the',
 'U.S.',
 ',',
 'you',
 'could',
 'use',
 'the',
 'Appraisal',
 'Subcommittee',
 '’',
 

In [22]:
def get_changed_word(ppd_match): 
    

In [103]:
from collections import Counter

def check_word_in_context(context, matches): 
    # count all the words in the context and make a dict representation of it 
    word_frequency = Counter()
    ppdb_matches = [elem[0][0] for elem in matches]

    
    tokenized = word_tokenize(context)
    for token in tokenized: 
        token = token.lower()
        word_frequency[token] += 1
    d = {}
    for ppdb_match in ppdb_matches:
        if ppdb_match.lower() in dict(word_frequency).keys(): 
            count_for_match = word_frequency[ppdb_match]
            if count_for_match > 1: 
                d[ppdb_match] = True
            else:
                d[ppdb_match] = False
    print(d)    

In [104]:
for i in range(10): 
    print(data[i]['PPDB_Matches'])
    check_word_in_context(data[i]['Source_Context_5_Processed'], data[i]['PPDB_Matches'])

[[['change', 'NN'], ['chance', 'NN']]]
{'change': False}
[[['changes', 'NNS'], ['chances', 'NNS']]]
{'changes': False}
[[['Humour', 'NNP'], ['Humor', 'NNP']]]
{'Humour': False}
[[['voltage', 'NN'], ['resistance', 'NN']]]
{'voltage': True}
[[['TP', 'NNP'], ['FP', 'NNP']]]
{'TP': False}
[[['Parent', 'NN'], ['Mother', 'NN']]]
{'Parent': False}
[[['click', 'NN'], ['tap', 'NN']]]
{'click': True}
[[['Maker', 'NNP'], ['Machine', 'NNP']]]
{'Maker': False}
[[['window', 'NN'], ['page', 'NN']]]
{'window': True}
[[['gentleman', 'NN'], ['gent', 'NN']]]
{'gentleman': True}


In [66]:
data[10]['Source_Context_5_Processed']

'To find an appraiser who might be able to train you, run a search online for “real estate appraisers” or some similar formulation. Alternately, use your phone book to locate real estate appraisers. * In some cases, your local licensing authority will connect you with an appraiser. * You can often begin as an apprentice before beginning your educational requirements, but you won’t be able to complete your apprenticeship and become a licensed real estate appraiser without gaining the requisite education. * In the U.S., you could use the Appraisal Subcommittee’s “Find an appraiser” search function at https://www.asc.gov/Pages/FindAnAppraiser.aspx. 3. Complete the real estate appraisal application. The application varies from country to country, and regional variations often exist even within a single nation. Generally, however, you’ll need to include (or agree to) a criminal background check, provide copies of your academic transcripts, and provide basic contact information (including yo

In [67]:
d = data[10]['PPDB_Matches']

In [68]:
d

[[['appraisal', 'NN'], ['appraiser', 'NN']]]

In [42]:
source_word = [pair[0][0] for elem in d for pair in elem]

In [43]:
source_word

['a', 'a']

In [54]:
source_words = [elem[0][0] for elem in example]
source_words

['thinking', 'outcomes']